In [ ]:
import dautil as dl
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import ch9util
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV
from IPython.display import HTML
import warnings

warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
X_train, X_test, y_train, y_test = ch9util.rain_split()
default = DecisionTreeClassifier(random_state=53, min_samples_leaf=3,
                                 max_depth=4)
entropy = DecisionTreeClassifier(criterion='entropy',
                                 min_samples_leaf=3, max_depth=4,
                                 random_state=57)
random = DecisionTreeClassifier(splitter='random', min_samples_leaf=3,
                                max_depth=4, random_state=5)

In [ ]:
clf = VotingClassifier([('default', default), 
                        ('entropy', entropy), ('random', random)])
params = {'voting': ['soft', 'hard'],
         'weights': [None, (2, 1, 1), (1, 2, 1), (1, 1, 2)]}
gscv = GridSearchCV(clf, param_grid=params, n_jobs=-1, cv=5)
gscv.fit(X_train, y_train)
votes = gscv.predict(X_test)

preds = []

for n, c in zip(['default', 'entropy', 'random'], 
             [default, entropy, random]):
    c.fit(X_train, y_train)
    preds.append(c.predict(X_test))
    ch9util.npy_save(n, preds[-1])
                 
preds = np.array(preds)

In [ ]:
%matplotlib inline
context = dl.nb.Context('stacking_multiple')
dl.nb.RcWidget(context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
html = ch9util.report_rain(votes, y_test, gscv.best_params_, sp.ax)
sp.ax.set_title(sp.ax.get_title() + ' | Voting')

preds_train = np.array([clf.predict(X_train)
                        for clf in [default, entropy, random]])

default.fit(preds_train.T, y_train)
stacked_preds = default.predict(preds.T)
ch9util.npy_save('stacking', stacked_preds)
html += ch9util.report_rain(stacked_preds, 
                            y_test, default.get_params(), sp.next_ax())
sp.ax.set_title(sp.ax.get_title() + ' | Stacking')

ch9util.plot_learn_curve(sp.next_ax(), gscv.best_estimator_, X_train,
                         y_train, title='Voting')

ch9util.plot_learn_curve(sp.next_ax(), default, X_train,
                         y_train, title='Stacking')
HTML(html + sp.exit())